In [3]:
import os
import json
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime
import re

In [4]:
# save path
save_path = r'F:\collection_spb_info\CLIMATE'

In [5]:
# define headers
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    'Host': 'www.ncei.noaa.gov',
    'Referer': 'https://www.ncei.noaa.gov/data/global-historical-climatology-network-daily/',
    'Sec-Ch-Ua': '"Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"',
    'Sec-Ch-Ua-Mobile': '?0',
    'Sec-Ch-Ua-Platform': '"Windows"',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'
}


In [6]:
# url 
host_url = 'https://www.ncei.noaa.gov/data/global-historical-climatology-network-daily/access/'

In [18]:
response = requests.get(host_url,headers=headers)

In [19]:

soup = BeautifulSoup(response.content, 'html.parser')

# 找到所有链接
links = soup.find_all('a', href=True)

# 正则表达式匹配以CHM和CHW开头的CSV文件
pattern = re.compile(r'^(CHM|CHW).+\.csv$')



In [20]:
# url list
url_list = []
for link in links:
    href = link['href']
    if pattern.match(href):
        file_url = host_url + href
        url_list.append(file_url)

In [ ]:
# download save
def download_file(file_url, headers, save_path, retries=3):
    for attempt in range(retries):
        try:
            file_response = requests.get(file_url, headers=headers, stream=True)
            file_name = os.path.join(save_path, os.path.basename(file_url))
            with open(file_name, 'wb') as file:
                for chunk in file_response.iter_content(chunk_size=512):
                    if chunk:
                        file.write(chunk)
            print(f'Saved {file_name}')
            return
        except (requests.exceptions.ChunkedEncodingError, requests.exceptions.ConnectionError) as e:
            print(f'Error downloading {file_name}: {e}. Retrying {attempt + 1}/{retries}...')
            time.sleep(2 ** attempt)  # 指数退避
    print(f'Failed to download {file_url} after {retries} attempts.')

# 下载所有文件
for file_url in url_list:
    print(f'Downloading {file_url}')
    download_file(file_url, headers, save_path)